In [ ]:
#!wget https://disk.skbkontur.ru/index.php/s/XPMEDq3Ko6tHJmb/download

In [ ]:
#!unzip download

In [1]:
import pandas
import numpy as np
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm

In [15]:
DEVICE = 'cuda:0'
EPOCHS = 30
BATCH_SIZE = 512

In [3]:
train = pandas.read_csv('train_data.tsv', sep='\t')

In [4]:
train_true = train[train['answer'] == True]
train_false = train[train['answer'] == False]

In [5]:
len(train), len(train_true), len(train_false)

(3983203, 182876, 3800327)

In [6]:
pos_weight =len(train_false) / len(train_true)
pos_weight

20.780895251427197

# Preprocess data

In [3]:
org_forms_ru = open('org.forms.ru', encoding='utf-8').read().split('\n')
org_forms_en = open('org.forms.en', encoding='utf-8').read().split('\n')

In [4]:
org_forms_ru = [x.lower() for x in org_forms_ru]
org_forms_en = [x.lower() for x in org_forms_en]

In [5]:
def find(s, ch = '\"'):
    return [i for i, i_ch in enumerate(s) if i_ch == ch]

In [6]:
def preprocess_rus(name):
    name = name.lower()
    quota_idx = find(name)
    if len(quota_idx) == 2 and quota_idx[1] - quota_idx[0] > 1:
        name = name[quota_idx[0] + 1 : quota_idx[1]]
    elif len(quota_idx) == 3 and quota_idx[2] - quota_idx[1] > 1:
        name = name[quota_idx[1] + 1 : quota_idx[2]]
    name = ' '.join([x for x in re.split('[ ,\-\"<>(){}:;*«»“”?!~%@#$%\\+_=^\[\]]', name) if x != ''])
    for form in org_forms_ru:
        if form in name:
            name = name.replace(form, '')
    return name.strip()

In [7]:
def preprocess_eng(name):
    name = name.lower()
    quota_idx = find(name)
    if len(quota_idx) == 2 and quota_idx[1] - quota_idx[0] > 1:
        name = name[quota_idx[0] + 1 : quota_idx[1]]
    elif len(quota_idx) == 3 and quota_idx[2] - quota_idx[1] > 1:
        name_tmp = name[quota_idx[1] + 1 : quota_idx[2]].strip()
        if name_tmp in org_forms_en:
            name = name[quota_idx[0] + 1 : quota_idx[1]]
        else:
            name = name_tmp
    name = ' '.join([x for x in re.split('[ ,\-\"<>(){}:;*«»“”?!~%@#$%\\+_=^\[\]]', name) if x != ''])
    for form in org_forms_en:
        if form in name:
            name = name.replace(form, '')
    return name.strip()

In [12]:
train['ru_name'] = train['ru_name'].apply(preprocess_rus)
train['eng_name'] = train['eng_name'].apply(preprocess_eng)

In [57]:
train_true = train[train['answer'] == True]
train_false = train[train['answer'] == False]

In [58]:
train_false = train_false.sample(11 * len(train_true))

### Make dataset proportion pos : neg = 1 : 10

In [59]:
false_size = 10 * len(train_true)
X = np.concatenate((train_false[:false_size][['ru_name', 'eng_name']], 
                    train_true[['ru_name', 'eng_name']]))
X = np.array([[ru.split(), en.split()] for ru, en in X])

y = np.concatenate((np.zeros(false_size, dtype=np.int), 
                    np.ones(len(train_true), dtype=np.int)))

In [60]:
X_neg = train_false.iloc[false_size:][['ru_name', 'eng_name']].values
X_neg = np.array([[ru.split(), en.split()] for ru, en in X_neg])

# Prepare model input

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01)

In [63]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.01)

### Make test and val set proportion pos : neg = 1 : pos_weight (20)

In [64]:
pos_cnt = sum(y_test)
prop = int(len(y_test) / pos_cnt + 0.5)
cnt = pos_cnt * (int(pos_weight) - prop)
prop, pos_cnt, cnt

(6, 1813, 25382)

In [65]:
X_test = np.concatenate((X_test, X_neg[:cnt]))
y_test = np.concatenate((y_test, np.zeros(cnt, dtype=np.int)))

In [66]:
X_neg = X_neg[cnt:]

In [67]:
pos_cnt = sum(y_val)
prop = int(len(y_val) / pos_cnt + 0.5)
cnt = pos_cnt * (int(pos_weight) - prop)
prop, pos_cnt, cnt

(6, 1843, 25802)

In [68]:
X_val = np.concatenate((X_val, X_neg[:cnt]))
y_val = np.concatenate((y_val, np.zeros(cnt, dtype=np.int)))

### Sort company names by sentence length, to reduce padding in batches

In [69]:
X_train, y_train = zip(*sorted(zip(X_train, y_train), key=lambda x: max(len(x[0][0]), len(x[0][1]))))
X_train = np.array(X_train)
y_train = np.array(y_train)

In [70]:
X_test, y_test = zip(*sorted(zip(X_test, y_test), key=lambda x: max(len(x[0][0]), len(x[0][1]))))
X_test = np.array(X_test)
y_test = np.array(y_test)

In [71]:
X_val, y_val = zip(*sorted(zip(X_val, y_val), key=lambda x: max(len(x[0][0]), len(x[0][1]))))
X_val = np.array(X_val)
y_val = np.array(y_val)

### Build vocabulary of english and russioan chars

In [72]:
def build_vocabs(arr):
    word_vocab = [word for word in set(np.concatenate(arr))]
    char_vocab = ['PAD', 'UNK'] + [c for c in set.union(*[set(word) for word in word_vocab])]
    char2id = {c: i for i, c in enumerate(char_vocab)}
    vocab = {'cv': char_vocab, 'c2i': char2id}
    return vocab

In [73]:
ru_vocab = build_vocabs(X[:, 0])

In [74]:
en_vocab = build_vocabs(X[:, 1])

In [75]:
len(en_vocab['cv']), len(ru_vocab['cv'])

(90, 88)

# Model

In [8]:
class CharEmbedding(nn.Module):
    def __init__(self,
                 vocab_size,
                 embedding_dim=32, 
                 hidden_dim=64,
                 k=5):

        super(CharEmbedding, self).__init__()
        self.embedder = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.cnn = nn.Conv1d(embedding_dim, hidden_dim, kernel_size=k, padding=k-1)
        
        
    def forward(self, sent):
        embeddings = self.embedder(sent)
        bs, sl, wl, ed = embeddings.shape
        embeddings = embeddings.view(bs * sl, wl, ed)
        embeddings = embeddings.transpose(2, 1)
        embeddings = self.cnn(embeddings)
        pool = nn.MaxPool1d(embeddings.size(-1))
        embeddings = pool(embeddings)
        embeddings = embeddings.view(bs, sl, -1)
        return embeddings

In [9]:
class FeedForward(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__() 
        self.linear = nn.Linear(in_dim, out_dim)
        self.dropout = nn.Dropout(p = 0.1)
    def forward(self, x):
        x = self.dropout(
            F.relu(
                self.linear(x)
            )
        )
        return x

In [10]:
class Model(nn.Module):

    def __init__(self,
                 ru_vocab_size, 
                 en_vocab_size,
                 embedding_dim=32,
                 embedding_hidden_dim=64,
                 hidden_dim=256,
                 k=5):

        super(Model, self).__init__()
        
        self.ru_embedder = CharEmbedding(ru_vocab_size, embedding_dim, embedding_hidden_dim, k)
        self.en_embedder = CharEmbedding(en_vocab_size, embedding_dim, embedding_hidden_dim, k)
        
        self.hidden_dim = hidden_dim
        
        self.ru_gru = nn.GRU(embedding_hidden_dim, hidden_dim, 
                              batch_first=True, bidirectional=True)
        self.en_gru = nn.GRU(embedding_hidden_dim, hidden_dim, 
                              batch_first=True, bidirectional=True)
        
        self.ru_ff = FeedForward(2 * hidden_dim, 4 * hidden_dim)
        self.en_ff = FeedForward(2 * hidden_dim, 4 * hidden_dim)
        
        self.final_ff = FeedForward(8 * hidden_dim, hidden_dim)
        self.linear = nn.Linear(hidden_dim, 1)
    
    def forward(self, ru_sent, en_sent):
        ru_embedding = self.ru_embedder(ru_sent)
        en_embedding = self.en_embedder(en_sent)
        
        h0 = torch.randn(2, ru_embedding.size(0), self.hidden_dim).to(DEVICE)
        _, ru_hidden = self.ru_gru(ru_embedding, h0)
        ru_hidden = ru_hidden.transpose(1, 0).contiguous()
        ru_hidden = ru_hidden.view(ru_hidden.size(0), -1)
        
        h0 = torch.randn(2, ru_embedding.size(0), self.hidden_dim).to(DEVICE)
        _, en_hidden = self.en_gru(en_embedding, h0)
        en_hidden = en_hidden.transpose(1, 0).contiguous()
        en_hidden = en_hidden.view(en_hidden.size(0), -1)
                
        ru = self.ru_ff(ru_hidden)
        en = self.en_ff(en_hidden)
        
        final = self.final_ff(torch.cat((ru, en), 1))
        return self.linear(final).view(-1)

In [11]:
def prepare_input(batch, c2i, **kwargs):
    char_batch = []
    max_words_len = 0
    for sentence in batch:
        max_words_len = max(max_words_len, len(sentence))
        char_batch.append([[c2i.get(c) or c2i['UNK'] for c in word] for word in sentence])
    max_chars_len = int(max(np.concatenate([[len(word) for word in sentence] for sentence in char_batch])))
    for sentence in char_batch:
        sentence += [[c2i['PAD']] * max_chars_len] * (max_words_len - len(sentence))
        for word in sentence:
            word += [c2i['PAD']] * (max_chars_len - len(word))
    return torch.LongTensor(char_batch)

In [16]:
def batch_generator(X, ru_vocab, en_vocab, y = None, batch_size=BATCH_SIZE):
    for batch_id in tqdm(range(0, len(X) // BATCH_SIZE + 1)):
        start = batch_id * BATCH_SIZE
        end = (batch_id + 1) * BATCH_SIZE
        ru_batch = prepare_input(X[start:end, 0], **ru_vocab).to(DEVICE)
        en_batch = prepare_input(X[start:end, 1], **en_vocab).to(DEVICE)
        if y is not None:
            target_batch = torch.tensor(y[start:end], dtype=torch.float).to(DEVICE)
            yield ru_batch, en_batch, target_batch
        else:
            yield ru_batch, en_batch

In [17]:
def get_classes(logits):
    pred = torch.sigmoid(logits)
    return (pred >= 0.5).int().tolist()

In [18]:
def predict(X, model, ru_vocab, en_vocab):
    res = []
    model.eval()
    for ru, en in batch_generator(X, ru_vocab, en_vocab):
        logits = model(ru, en)
        res = np.concatenate((res, get_classes(logits)))
    return res

In [19]:
def evaluate(X, y, model, ru_vocab, en_vocab, loss_fnc, score_fnc):
    classes = []
    batch_losses = []
    model.eval()
    for ru, en, trg in batch_generator(X, ru_vocab, en_vocab, y):
        logits = model(ru, en)
        batch_losses.append(loss_fnc(logits, trg).tolist())
        classes = np.concatenate((classes, get_classes(logits)))
    loss = np.average(batch_losses)
    score = score_fnc(y, classes)
    return loss, score

# Train

In [20]:
from sklearn.metrics import f1_score

In [21]:
loss_fnc = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(DEVICE))

In [107]:
model = Model(len(ru_vocab['cv']), len(en_vocab['cv'])).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.0005)

In [108]:
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

In [109]:
from torch.utils.tensorboard import SummaryWriter

In [110]:
writer = SummaryWriter()

In [111]:
def train_model(model, X, y, ru_vocab, en_vocab, X_val, y_val, X_test, y_test, epochs = EPOCHS, start_epoch = 1):
    train_losses = []
    val_losses = []
    train_scores = []
    val_scores = []
    for epoch in range(start_epoch, epochs + 1):
        print(f'Epoch: {epoch} start')
        model.train()
        batch_losses = []
        batch_scores = []
        for i, (ru, en, trg) in enumerate(batch_generator(X, ru_vocab, en_vocab, y)):

            model.zero_grad()

            logits = model(ru, en)
            loss = loss_fnc(logits, trg)
            loss.backward()
            optimizer.step()

            batch_losses.append(loss.tolist())
            score = f1_score(trg.data.cpu(), get_classes(logits))
            batch_scores.append(score.tolist())

            if i % 100 == 0:
                print(f'Iter: {i}, loss: {loss}, score: {score}')

        epoch_loss = np.average(batch_losses)
        epoch_score = np.average(batch_scores)
        train_losses.append(epoch_loss)
        train_scores.append(epoch_score)
        
        writer.add_scalar('Loss/train', epoch_loss, epoch)
        writer.add_scalar('F1/train', epoch_score, epoch)
        
        val_loss, val_score = evaluate(X_val, y_val, model, ru_vocab, en_vocab, loss_fnc, f1_score)
        
        writer.add_scalar('Loss/val', val_loss, epoch)
        writer.add_scalar('F1/val', val_score, epoch)
        
        test_loss, test_score = evaluate(X_test, y_test, model, ru_vocab, en_vocab, loss_fnc, f1_score)
        
        writer.add_scalar('Loss/test', test_loss, epoch)
        writer.add_scalar('F1/test', test_score, epoch)
        
        val_losses.append(val_loss)
        val_scores.append(val_score)
        
        print(f'Epoch: {epoch} end')
        print(f'train_loss: {epoch_loss}, train_score: {epoch_score}')
        print(f'val_loss: {val_loss}, val_score: {val_score}')
        print(f'test_loss: {test_loss}, test_score: {test_score}')
        
        if len(val_scores) < 2 or val_score > max(val_scores[:-1]):
            torch.save((ru_vocab, en_vocab, model.state_dict()), 
                       f'model/chpt-loss{val_loss:.{2}f}-score{val_score:.{2}f}.pt')
        
            
    return train_losses, val_losses, train_scores, val_scores

In [112]:
hist = train_model(model, X_train, y_train, ru_vocab, en_vocab, X_val, y_val, X_test, y_test, epochs=60, start_epoch=1)

Epoch: 1 start


Iter: 0, loss: 1.5463892221450806, score: 0.3673469387755102
Iter: 100, loss: 1.3486206531524658, score: 0.40498442367601245
Iter: 200, loss: 1.4298536777496338, score: 0.49189985272459497
Iter: 300, loss: 1.3609228134155273, score: 0.4318529862174579
Iter: 400, loss: 1.403918743133545, score: 0.4807121661721068
Iter: 500, loss: 1.1457114219665527, score: 0.2614601018675722
Iter: 600, loss: 0.8024235963821411, score: 0.46280991735537186
Iter: 700, loss: 0.7790716886520386, score: 0.467741935483871
Iter: 800, loss: 0.8403072357177734, score: 0.38135593220338987
Iter: 900, loss: 0.6771206855773926, score: 0.6379310344827587
Iter: 1000, loss: 0.5977673530578613, score: 0.5666666666666667
Iter: 1100, loss: 0.6626445055007935, score: 0.5934065934065933
Iter: 1200, loss: 0.6040080785751343, score: 0.5212121212121212
Iter: 1300, loss: 0.583320140838623, score: 0.5838509316770186
Iter: 1400, loss: 0.5843689441680908, score: 0.6473988439306357
Iter: 1500, loss: 0.6661043167114258, score: 0.6393


Epoch: 1 end
train_loss: 0.8840023560191494, train_score: 0.4990903917567779
val_loss: 2.6423793766233654, val_score: 0.09659225058369841
test_loss: 2.600134390923712, test_score: 0.09564664760088974
Epoch: 2 start


Iter: 0, loss: 1.9069973230361938, score: 0.441400304414003
Iter: 100, loss: 0.9047029614448547, score: 0.6490765171503957
Iter: 200, loss: 0.8237023949623108, score: 0.7386091127098321
Iter: 300, loss: 0.6676536202430725, score: 0.7282321899736147
Iter: 400, loss: 0.7263509035110474, score: 0.7661691542288558
Iter: 500, loss: 0.922964334487915, score: 0.5876288659793815
Iter: 600, loss: 0.5880406498908997, score: 0.6419753086419754
Iter: 700, loss: 0.42066383361816406, score: 0.738255033557047
Iter: 800, loss: 0.7019368410110474, score: 0.5657894736842105
Iter: 900, loss: 0.506339430809021, score: 0.7643979057591622
Iter: 1000, loss: 0.4330361485481262, score: 0.6624203821656051
Iter: 1100, loss: 0.5009876489639282, score: 0.7088607594936709
Iter: 1200, loss: 0.41778305172920227, score: 0.6178861788617885
Iter: 1300, loss: 0.443946897983551, score: 0.7194244604316549
Iter: 1400, loss: 0.48939618468284607, score: 0.718954248366013
Iter: 1500, loss: 0.6300699710845947, score: 0.59999999


Epoch: 2 end
train_loss: 0.6459640135145482, train_score: 0.6443226867080046
val_loss: 1.367750595841143, val_score: 0.12795312094351521
test_loss: 1.353127404840456, test_score: 0.12789044376262815
Epoch: 3 start


...


Epoch: 58 end
train_loss: 0.0567632564675969, train_score: 0.9602737813353003
val_loss: 0.35799362510442734, val_score: 0.6658791100610357
test_loss: 0.3441003653117352, test_score: 0.6518161749949829
Epoch: 59 start


Iter: 0, loss: 0.07737132906913757, score: 0.9727891156462585
Iter: 100, loss: 0.050004955381155014, score: 0.9772727272727273
Iter: 200, loss: 0.09610086679458618, score: 0.973293768545994
Iter: 300, loss: 0.0864306390285492, score: 0.958904109589041
Iter: 400, loss: 0.11708296090364456, score: 0.9724770642201834
Iter: 500, loss: 0.05355362594127655, score: 0.9565217391304348
Iter: 600, loss: 0.048659853637218475, score: 0.948905109489051
Iter: 700, loss: 0.03215913102030754, score: 0.9640287769784173
Iter: 800, loss: 0.05257318168878555, score: 0.9586776859504132
Iter: 900, loss: 0.030329829081892967, score: 0.9775280898876404
Iter: 1000, loss: 0.02988128364086151, score: 0.9682539682539683
Iter: 1100, loss: 0.03511440381407738, score: 0.948905109489051
Iter: 1200, loss: 0.0319434255361557, score: 0.9523809523809523
Iter: 1300, loss: 0.04732336476445198, score: 0.9523809523809523
Iter: 1400, loss: 0.0956290140748024, score: 0.9701492537313433
Iter: 1500, loss: 0.0373106524348259, sco


Epoch: 59 end
train_loss: 0.057359028505210165, train_score: 0.9605265581697007
val_loss: 0.35178324456016224, val_score: 0.6437654203833745
test_loss: 0.3547269912712573, test_score: 0.6213444739840486
Epoch: 60 start


Iter: 0, loss: 0.05002360790967941, score: 0.9797297297297297
Iter: 100, loss: 0.06415273249149323, score: 0.962962962962963
Iter: 200, loss: 0.0897132009267807, score: 0.9702380952380952
Iter: 300, loss: 0.05299224704504013, score: 0.962457337883959
Iter: 400, loss: 0.03972931206226349, score: 0.9787234042553192
Iter: 500, loss: 0.04764987528324127, score: 0.9743589743589742
Iter: 600, loss: 0.03809270262718201, score: 0.962962962962963
Iter: 700, loss: 0.07173996418714523, score: 0.962962962962963
Iter: 800, loss: 0.08169574290513992, score: 0.9206349206349207
Iter: 900, loss: 0.09110546857118607, score: 0.9828571428571428
Iter: 1000, loss: 0.03994207829236984, score: 0.9465648854961832
Iter: 1100, loss: 0.044915568083524704, score: 0.962962962962963
Iter: 1200, loss: 0.06478199362754822, score: 0.9423076923076924
Iter: 1300, loss: 0.03815505653619766, score: 0.9600000000000001
Iter: 1400, loss: 0.04071201756596565, score: 0.9571428571428571
Iter: 1500, loss: 0.05566524341702461, sco


Epoch: 60 end
train_loss: 0.053556369364470856, train_score: 0.9624972171964077
val_loss: 0.37191666817913455, val_score: 0.6771399798590131
test_loss: 0.3641357294594248, test_score: 0.6632715420655938


# Load model and predict

In [25]:
test = pandas.read_csv('test_data.tsv', sep='\t')

In [28]:
test['ru_name'] = test['ru_name'].apply(preprocess_rus)
test['eng_name'] = test['eng_name'].apply(preprocess_eng)

In [29]:
X = test[['ru_name', 'eng_name']].values

In [30]:
X = np.array([[ru.split(), en.split()] for ru, en in X])

In [31]:
ru_vocab, en_vocab, state_dict = torch.load('model/chpt-loss0.51-score0.70.pt')

In [32]:
model = Model(len(ru_vocab['cv']), len(en_vocab['cv'])).to(DEVICE)

In [33]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [34]:
y = predict(X, model, ru_vocab, en_vocab)

In [35]:
y = np.array(y, dtype = np.bool)

In [36]:
sum(y), sum(y) / (len(y) - sum(y))

(24307, 0.025013763898965265)

In [37]:
res = pandas.DataFrame(y, columns=['answer'])

In [38]:
res.to_csv('result.tsv', index=False)